In [82]:
def sql_to_mongo(sql_query):
    """
    Convert SQL queries to MongoDB aggregation pipelines.
    """
    sql_query = sql_query.strip()
    
    # 1
    # Handle SELECT * FROM table_name LIMIT 5
    match_limit = re.match(r"SELECT\s+\*\s+FROM\s+(\w+)\s+LIMIT\s+(\d+)", sql_query, re.IGNORECASE)
    if match_limit:
        collection, limit = match_limit.groups()
        return f"db.{collection}.aggregate([{{ '$limit': {int(limit)} }}]);"

    # 2, 6
    # Handle SELECT * FROM table_name ORDER BY column_name ASC|DESC LIMIT 5
    match_order_limit = re.match(
        r"SELECT\s+\*\s+FROM\s+(\w+)\s+ORDER\s+BY\s+(\w+)\s*(ASC|DESC)?\s+LIMIT\s+(\d+)", 
        sql_query, re.IGNORECASE
    )
    if match_order_limit:
        collection, column_name, order, limit = match_order_limit.groups()
        sort_order = 1 if order and order.upper() == "ASC" else -1
        return f"db.{collection}.aggregate([{{ '$sort': {{ '{column_name}': {sort_order} }} }}, {{ '$limit': {int(limit)} }}]);"
    
    # 3
    # Handle SELECT COUNT(*) FROM table_name
    match_count_all = re.match(r"SELECT\s+COUNT\(\*\)\s+FROM\s+(\w+)", sql_query, re.IGNORECASE)
    if match_count_all:
        collection = match_count_all.group(1)
        return f"db.{collection}.aggregate([{{ '$count': 'total_count' }}]);"
    

    #4
    # Handle SELECT * FROM table_name WHERE column_name BETWEEN 1 AND 3
    match_between = re.match(r"SELECT\s+\*\s+FROM\s+(\w+)\s+WHERE\s+(\w+)\s+BETWEEN\s+(\d+)\s+AND\s+(\d+)", sql_query, re.IGNORECASE)
    if match_between:
        collection, column_name, start, end = match_between.groups()
        return (
            f"db.{collection}.aggregate([{{ '$match': {{ '{column_name}': {{ '$gte': {int(start)}, '$lte': {int(end)} }} }} }}]);"
        )
    
    # 5
    # Handle SELECT * FROM table_name WHERE column_name != '...'
    match_not_equal = re.match(r"SELECT\s+\*\s+FROM\s+(\w+)\s+WHERE\s+(\w+)\s+!=\s+(.+?)", sql_query, re.IGNORECASE)
    if match_not_equal:
        collection, column_name, value = match_not_equal.groups()
        return f"db.{collection}.aggregate([{{ '$match': {{ '{column_name}': {{ '$ne': {value} }} }} }}]);"
    
    # 7
    # Handle SELECT * FROM table_name WHERE column_name IS NULL
    match_is_null = re.match(r"SELECT\s+\*\s+FROM\s+(\w+)\s+WHERE\s+(\w+)\s+IS\s+NULL", sql_query, re.IGNORECASE)
    if match_is_null:
        collection, column_name = match_is_null.groups()
        return f"db.{collection}.aggregate([{{ '$match': {{ '{column_name}': {{ '$exists': true, '$eq': null }} }} }}]);"

    # 8
    # Handle SELECT * FROM table_name WHERE column_name IS NOT NULL
    match_is_not_null = re.match(r"SELECT\s+\*\s+FROM\s+(\w+)\s+WHERE\s+(\w+)\s+IS\s+NOT\s+NULL", sql_query, re.IGNORECASE)
    if match_is_not_null:
        collection, column_name = match_is_not_null.groups()
        return f"db.{collection}.aggregate([{{ '$match': {{ '{column_name}': {{ '$exists': true, '$ne': null }} }} }}]);"
    
    
    # 1 - 8
    
    ####################
    
    
    # 9
    # Handle SELECT * FROM table_name WHERE column_name LIKE '%pattern%'
    # Handle SELECT * FROM table_name WHERE column_name LIKE '%pattern%'
    # Handle SELECT * FROM table_name WHERE column_name IN (value1, value2, value3)
    # Handle SELECT * FROM table_name WHERE column_name IN (value1, value2, value3)
    match_in = re.match(
        r"SELECT\s+\*\s+FROM\s+(\w+)\s+WHERE\s+(\w+)\s+IN\s+\((.+)\)", 
        sql_query.strip(), 
        re.IGNORECASE
    )
    if match_in:
        collection, column_name, values = match_in.groups()
        value_list = [v.strip() for v in values.split(",")]  # Split values and trim whitespace
        # Convert numeric values to integers if possible
        value_list = [int(v) if v.isdigit() else v.strip("'\"") for v in value_list]
        return f"db.{collection}.aggregate([{{ '$match': {{ '{column_name}': {{ '$in': {value_list} }} }} }}]);"
    
    # 10
    # Handle SELECT column_name, COUNT(aggregate_column) FROM table_name GROUP BY group_column
    match_group_by = re.match(
        r"SELECT\s+(\w+)\s*,\s*COUNT\((\w+)\)\s+FROM\s+(\w+)\s+GROUP\s+BY\s+(\w+)", 
        sql_query.strip(), 
        re.IGNORECASE
    )
    if match_group_by:
        group_column, aggregate_column, collection, group_by_column = match_group_by.groups()
        return (
            f"db.{collection}.aggregate(["
            f"{{ '$group': {{ '_id': '${group_column}', 'count': {{ '$sum': 1 }} }} }},"
            f"{{ '$project': {{ '{group_column}': '$_id', '_id': 0, 'count': 1 }} }}"
            f"]);"
        )

    # 11
    # Handle SELECT column_name, AVG(aggregate_column) FROM table_name GROUP BY group_column
    match_group_avg = re.match(
        r"SELECT\s+(\w+)\s*,\s*AVG\((\w+)\)\s+FROM\s+(\w+)\s+GROUP\s+BY\s+(\w+)", 
        sql_query.strip(), 
        re.IGNORECASE
    )
    if match_group_avg:
        group_column, aggregate_column, collection, group_by_column = match_group_avg.groups()
        return (
            f"db.{collection}.aggregate(["
            f"{{ '$group': {{ '_id': '${group_column}', 'avg_salary': {{ '$avg': '${aggregate_column}' }} }} }},"
            f"{{ '$project': {{ '{group_column}': '$_id', '_id': 0, 'avg_salary': 1 }} }}"
            f"]);"
        )

    
    
    # TEMPLATES FOR JOINS
    
    # 1
    
    # Handle SELECT with JOIN
    match_join = re.match(
        r"SELECT\s+(\w+\.\w+),\s+(\w+\.\w+)\s+FROM\s+(\w+)\s+JOIN\s+(\w+)\s+ON\s+(\w+\.\w+)\s*=\s*(\w+\.\w+)\s+WHERE\s+(\w+)\s*=\s*'(.+)'",
        sql_query.strip(),
        re.IGNORECASE
    )
    if match_join:
        select_field1, select_field2, collection1, collection2, join_field1, join_field2, filter_field, filter_value = match_join.groups()
        return (
            f"db.{collection1}.aggregate(["
            f"{{ '$lookup': {{ 'from': '{collection2}', 'localField': '{join_field1.split('.')[1]}', 'foreignField': '{join_field2.split('.')[1]}', 'as': 'joined_docs' }} }},"
            f"{{ '$unwind': '$joined_docs' }},"
            f"{{ '$match': {{ '{filter_field}': '{filter_value}' }} }},"
            f"{{ '$project': {{ '{select_field1.split('.')[1]}': 1, '{select_field2.split('.')[1]}': '$joined_docs.{select_field2.split('.')[1]}', '_id': 0 }} }}"
            f"]);"
        )

    # 3
    # Handle SELECT with JOIN and IS NULL condition
    match_join_is_null = re.match(
        r"SELECT\s+(\w+\.\w+),\s+(\w+\.\w+)\s+FROM\s+(\w+)\s+JOIN\s+(\w+)\s+ON\s+(\w+\.\w+)\s*=\s*(\w+\.\w+)\s+WHERE\s+(\w+)\s+IS\s+NULL",
        sql_query.strip(),
        re.IGNORECASE
    )
    if match_join_is_null:
        select_field1, select_field2, collection1, collection2, join_field1, join_field2, filter_field = match_join_is_null.groups()
        return (
            f"db.{collection1}.aggregate(["
            f"{{ '$lookup': {{ 'from': '{collection2}', 'localField': '{join_field1.split('.')[1]}', 'foreignField': '{join_field2.split('.')[1]}', 'as': 'joined_docs' }} }},"
            f"{{ '$unwind': '$joined_docs' }},"
            f"{{ '$match': {{ '{filter_field}': {{ '$exists': true, '$eq': null }} }} }},"
            f"{{ '$project': {{ '{select_field1.split('.')[1]}': 1, '{select_field2.split('.')[1]}': '$joined_docs.{select_field2.split('.')[1]}', '_id': 0 }} }}"
            f"]);"
        )

    #4
    # Handle SELECT with JOIN and IS NOT NULL condition
    match_join_is_not_null = re.match(
        r"SELECT\s+(\w+\.\w+),\s+(\w+\.\w+)\s+FROM\s+(\w+)\s+JOIN\s+(\w+)\s+ON\s+(\w+\.\w+)\s*=\s*(\w+\.\w+)\s+WHERE\s+(\w+)\s+IS\s+NOT\s+NULL",
        sql_query.strip(),
        re.IGNORECASE
    )
    if match_join_is_not_null:
        select_field1, select_field2, collection1, collection2, join_field1, join_field2, filter_field = match_join_is_not_null.groups()
        return (
            f"db.{collection1}.aggregate(["
            f"{{ '$lookup': {{ 'from': '{collection2}', 'localField': '{join_field1.split('.')[1]}', 'foreignField': '{join_field2.split('.')[1]}', 'as': 'joined_docs' }} }},"
            f"{{ '$unwind': '$joined_docs' }},"
            f"{{ '$match': {{ '{filter_field}': {{ '$exists': true, '$ne': null }} }} }},"
            f"{{ '$project': {{ '{select_field1.split('.')[1]}': 1, '{select_field2.split('.')[1]}': '$joined_docs.{select_field2.split('.')[1]}', '_id': 0 }} }}"
            f"]);"
        )
    
    ########
    
    #5
    # Handle SELECT with JOIN and IN condition
    match_join_in = re.match(
        r"SELECT\s+(\w+\.\w+),\s+(\w+\.\w+)\s+FROM\s+(\w+)\s+JOIN\s+(\w+)\s+ON\s+(\w+\.\w+)\s*=\s*(\w+\.\w+)\s+WHERE\s+(\w+)\s+IN\s+\((.+)\)",
        sql_query.strip(),
        re.IGNORECASE
    )
    if match_join_in:
        select_field1, select_field2, collection1, collection2, join_field1, join_field2, filter_field, filter_values = match_join_in.groups()
        value_list = [v.strip().strip("'\"") for v in filter_values.split(",")]  # Split and clean values
        return (
            f"db.{collection1}.aggregate(["
            f"{{ '$lookup': {{ 'from': '{collection2}', 'localField': '{join_field1.split('.')[1]}', 'foreignField': '{join_field2.split('.')[1]}', 'as': 'joined_docs' }} }},"
            f"{{ '$unwind': '$joined_docs' }},"
            f"{{ '$match': {{ '{filter_field}': {{ '$in': {value_list} }} }} }},"
            f"{{ '$project': {{ '{select_field1.split('.')[1]}': 1, '{select_field2.split('.')[1]}': '$joined_docs.{select_field2.split('.')[1]}', '_id': 0 }} }}"
            f"]);"
        )
    
    #6
    # Handle SELECT with JOIN and NOT IN condition
    match_join_not_in = re.match(
        r"SELECT\s+(\w+\.\w+),\s+(\w+\.\w+)\s+FROM\s+(\w+)\s+JOIN\s+(\w+)\s+ON\s+(\w+\.\w+)\s*=\s*(\w+\.\w+)\s+WHERE\s+(\w+)\s+NOT\s+IN\s+\((.+)\)",
        sql_query.strip(),
        re.IGNORECASE
    )
    if match_join_not_in:
        select_field1, select_field2, collection1, collection2, join_field1, join_field2, filter_field, filter_values = match_join_not_in.groups()
        value_list = [v.strip().strip("'\"") for v in filter_values.split(",")]  # Split and clean values
        return (
            f"db.{collection1}.aggregate(["
            f"{{ '$lookup': {{ 'from': '{collection2}', 'localField': '{join_field1.split('.')[1]}', 'foreignField': '{join_field2.split('.')[1]}', 'as': 'joined_docs' }} }},"
            f"{{ '$unwind': '$joined_docs' }},"
            f"{{ '$match': {{ '{filter_field}': {{ '$nin': {value_list} }} }} }},"
            f"{{ '$project': {{ '{select_field1.split('.')[1]}': 1, '{select_field2.split('.')[1]}': '$joined_docs.{select_field2.split('.')[1]}', '_id': 0 }} }}"
            f"]);"
        )




        # Raise error for unsupported queries
    print(f"Unsupported query format: {sql_query}")
    raise ValueError("Unsupported query format.")


In [83]:
print('1')
# 1. 
sql_query = """
SELECT *
FROM table_name
LIMIT 5;
"""
mongo_query = sql_to_mongo(sql_query)
print(mongo_query)

print('2')
# 2. 
sql_query = """
SELECT *
FROM table_name
ORDER BY column_name 
LIMIT 5;
"""
mongo_query = sql_to_mongo(sql_query)
print(mongo_query)

print('3')
# 3. 
sql_query = """
SELECT COUNT(*) 
FROM table_name;
"""
mongo_query = sql_to_mongo(sql_query)
print(mongo_query)

print('4')
# 4. 
sql_query = """
SELECT *
FROM table_name
WHERE column_name BETWEEN 1 AND 3;
"""
mongo_query = sql_to_mongo(sql_query)
print(mongo_query)

print('5')
# 5. 
sql_query = """
SELECT *
FROM table_name
WHERE column_name != 5;
"""
mongo_query = sql_to_mongo(sql_query)
print(mongo_query)

print('6')
# 6
sql_query = """
SELECT *
FROM table_name
LIMIT 5;
"""
mongo_query = sql_to_mongo(sql_query)
print(mongo_query)

print('7')
# 7
sql_query = """
SELECT *
FROM table_name
WHERE column_name IS NULL;
"""
mongo_query = sql_to_mongo(sql_query)
print(mongo_query)

print('8')
# 8 
sql_query = """
SELECT *
FROM table_name
WHERE column_name IS NOT NULL;
"""
mongo_query = sql_to_mongo(sql_query)
print(mongo_query)

print('9')
sql_query = """
SELECT *
FROM table_name
WHERE column_name IN (A, C, 3, 4);
"""
mongo_query = sql_to_mongo(sql_query)
print(mongo_query)

print('10')
sql_query = """
SELECT a, COUNT(b)
FROM table 
GROUP BY a;
"""
mongo_query = sql_to_mongo(sql_query)
print(mongo_query)

print('11')
sql_query = """
SELECT department, AVG(salary) 
FROM employees 
GROUP BY department;
"""
mongo_query = sql_to_mongo(sql_query)
print(mongo_query)

print('\njoins')

print('1')
sql_query = """
SELECT enrollments.Grade, students.Major
FROM enrollments
JOIN students ON enrollments.StudentID = students.StudentID
WHERE Grade = 'A';
"""
mongo_query = sql_to_mongo(sql_query)
print(mongo_query)

print('3')
sql_query = """
SELECT enrollments.Grade, courses.CourseName
FROM enrollments
JOIN courses ON enrollments.CourseID = courses.CourseID
WHERE Grade IS NULL;
"""
mongo_query = sql_to_mongo(sql_query)
print(mongo_query)

print('4')
sql_query = """
SELECT enrollments.Grade, courses.CourseName
FROM enrollments
JOIN courses ON enrollments.CourseID = courses.CourseID
WHERE Grade IS NOT NULL;
"""
mongo_query = sql_to_mongo(sql_query)
print(mongo_query)

# print('5')
# sql_query = """
# SELECT enrollments.Grade, courses.CourseName
# FROM enrollments
# JOIN courses ON enrollments.CourseID = courses.CourseID
# WHERE Grade IS NOT NULL;
# """
# mongo_query = sql_to_mongo(sql_query)
# print(mongo_query)

# print('6')
# sql_query = """
# SELECT enrollments.Grade, courses.CourseName
# FROM enrollments
# JOIN courses ON enrollments.CourseID = courses.CourseID
# WHERE Grade is NOT IN ('A', 'B', 'C');
# """
# mongo_query = sql_to_mongo(sql_query)
# print(mongo_query)

1
db.table_name.aggregate([{ '$limit': 5 }]);
2
db.table_name.aggregate([{ '$sort': { 'column_name': -1 } }, { '$limit': 5 }]);
3
db.table_name.aggregate([{ '$count': 'total_count' }]);
4
db.table_name.aggregate([{ '$match': { 'column_name': { '$gte': 1, '$lte': 3 } } }]);
5
db.table_name.aggregate([{ '$match': { 'column_name': { '$ne': 5 } } }]);
6
db.table_name.aggregate([{ '$limit': 5 }]);
7
db.table_name.aggregate([{ '$match': { 'column_name': { '$exists': true, '$eq': null } } }]);
8
db.table_name.aggregate([{ '$match': { 'column_name': { '$exists': true, '$ne': null } } }]);
9
db.table_name.aggregate([{ '$match': { 'column_name': { '$in': ['A', 'C', 3, 4] } } }]);
10
db.table.aggregate([{ '$group': { '_id': '$a', 'count': { '$sum': 1 } } },{ '$project': { 'a': '$_id', '_id': 0, 'count': 1 } }]);
11
db.employees.aggregate([{ '$group': { '_id': '$department', 'avg_salary': { '$avg': '$salary' } } },{ '$project': { 'department': '$_id', '_id': 0, 'avg_salary': 1 } }]);

joins
1
db.e

In [70]:
# 1 db.table_name.aggregate([{ '$limit': 5 }]);
# 2 db.table_name.aggregate([{ '$sort': { 'column_name': -1 } }, { '$limit': 5 }]);
# 3 db.table_name.aggregate([{ '$count': 'total_count' }]);
# 4 db.table_name.aggregate([{ '$match': { 'column_name': { '$gte': 1, '$lte': 3 } } }]);
# 5 db.table_name.aggregate([{ '$match': { 'column_name': { '$ne': 5 } } }]); 
# 6 db.table_name.aggregate([{ '$limit': 5 }]);
# 7 db.table_name.aggregate([{ '$match': { 'column_name': { '$exists': true, '$eq': null } } }]);
# 8 db.table_name.aggregate([{ '$match': { 'column_name': { '$exists': true, '$ne': null } } }]);
# 9 db.table_name.aggregate([{ '$match': { 'column_name': { '$in': ['A', 'C', 3, 4] } } }]);
# 10 db.employees.aggregate([{ '$group': { '_id': '$department', 'count': { '$sum': 1 } } },
# { '$project': { 'department': '$_id', '_id': 0, 'count': 1 } }]);
# 11 db.employees.aggregate([{ '$group': { '_id': '$department', 'avg_salary': { '$avg': '$salary' } } },
# { '$project': { 'department': '$_id', '_id': 0, 'avg_salary': 1 } }]);
# 12



In [ ]:
specifics:
# 7 db.table_name.aggregate([{ '$match': { 'column_name': { '$exists': true, '$eq': null } } }]);
# had to add exists here because in sql its different